In [ ]:
# Cloud Shell
gcloud auth list

In [ ]:
gcloud config list project

##Task 1. Proxy the Cloud Translation API

In [ ]:
# APIs & Services -> Library -> Cloud Translation API -> API Enabled

In [ ]:
# IAM & Admin -> Service Accounts -> + Create Service Account

In [ ]:
# Apigee -> API Proxies -> + Create -> ... -> Develop -> Target Endpoints | default ...

In [ ]:
# Compute Engine -> VM instances -> ... | SSH

export INSTANCE_NAME=eval-instance; export ENV_NAME=eval; export PREV_INSTANCE_STATE=; echo "waiting for runtime instance ${INSTANCE_NAME} to be active"; while : ; do export INSTANCE_STATE=$(curl -s -H "Authorization: Bearer $(gcloud auth print-access-token)" -X GET "https://apigee.googleapis.com/v1/organizations/${GOOGLE_CLOUD_PROJECT}/instances/${INSTANCE_NAME}" | jq "select(.state != null) | .state" --raw-output); [[ "${INSTANCE_STATE}" == "${PREV_INSTANCE_STATE}" ]] || (echo; echo "INSTANCE_STATE=${INSTANCE_STATE}"); export PREV_INSTANCE_STATE=${INSTANCE_STATE}; [[ "${INSTANCE_STATE}" != "ACTIVE" ]] || break; echo -n "."; sleep 5; done; echo; echo "instance created, waiting for environment ${ENV_NAME} to be attached to instance"; while : ; do export ATTACHMENT_DONE=$(curl -s -H "Authorization: Bearer $(gcloud auth print-access-token)" -X GET "https://apigee.googleapis.com/v1/organizations/${GOOGLE_CLOUD_PROJECT}/instances/${INSTANCE_NAME}/attachments" | jq "select(.attachments != null) | .attachments[] | select(.environment == \"${ENV_NAME}\") | .environment" --join-output); [[ "${ATTACHMENT_DONE}" != "${ENV_NAME}" ]] || break; echo -n "."; sleep 5; done; echo "***ORG IS READY TO USE***";


In [ ]:
# ... Save -> Deploy
# apigee-proxy@qwiklabs-gcp-02-499b66e75c7f.iam.gserviceaccount.com

In [ ]:
# SSH

TEST_VM_ZONE=$(gcloud compute instances list --filter="name=('apigeex-test-vm')" --format "value(zone)")
gcloud compute ssh apigeex-test-vm --zone=${TEST_VM_ZONE} --force-key-file-overwrite

In [ ]:
curl -i -k -X POST "https://eval.example.com/translate/v1" -H "Content-Type: application/json" -d '{ "q": "Translate this text!", "target": "es" }'


##Task 2. Change the API request and response

In [ ]:
# Develop -> Resources +: [Property set], [language.properties] -> XML:
output=es
caller=en

In [ ]:
# Develop -> Proxy Endpoints -> default +: [translate], [Path and Verb], [/], [POST]

In [ ]:
# translate | Request + [Mediation: Assign Message] -> AM-BuildTranslateRequest -> XML:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<AssignMessage name="AM-BuildTranslateRequest">
  <AssignVariable>
    <Name>text</Name>
    <Template>{jsonPath($.text,request.content)}</Template>
  </AssignVariable>
  <AssignVariable>
    <Name>language</Name>
    <Template>{firstnonnull(request.queryparam.lang,propertyset.language.output)}</Template>
  </AssignVariable>
  <Set>
    <Payload>{ "q": "{text}", "target": "{language}" }</Payload>
  </Set>
  <IgnoreUnresolvedVariables>true</IgnoreUnresolvedVariables>
  <AssignTo createNew="false" transport="http" type="request"/>
</AssignMessage>

In [ ]:
# translate | Response + [Mediation: Assign Message] -> AM-BuildTranslateResponse -> XML:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<AssignMessage name="AM-BuildTranslateResponse">
  <AssignVariable>
    <Name>translated</Name>
    <Template>{jsonPath($.data.translations[0].translatedText,response.content)}</Template>
  </AssignVariable>
  <Set>
    <Headers>
      <Header name="Content-Type">application/json</Header>
    </Headers>
    <Payload>{"translated": "{translated}" }</Payload>
  </Set>
  <IgnoreUnresolvedVariables>true</IgnoreUnresolvedVariables>
  <AssignTo createNew="true" transport="http" type="response"/>
</AssignMessage>

In [ ]:
# Develop -> Proxy Endpoints -> default +: [getLanguages], [Path], [/languages]

In [ ]:
# getLanguages | Request + [Mediation: Assign Message] -> AM-BuildLanguagesRequest -> XML:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<AssignMessage name="AM-BuildLanguagesRequest">
  <Set>
    <Verb>POST</Verb>
    <Payload contentType="application/json">{ "target": "{propertyset.language.caller}" }</Payload>
  </Set>
  <IgnoreUnresolvedVariables>true</IgnoreUnresolvedVariables>
  <AssignTo createNew="true" transport="http" type="request"/>
</AssignMessage>

In [ ]:
# getLanguages | Response + [Extension: JavaScript] -> JS-BuildLanguagesResponse -> XML:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<Javascript continueOnError="false" enabled="true" timeLimit="200" name="JS-BuildLanguagesResponse">
  <ResourceURL>jsc://buildLanguagesResponse.js</ResourceURL>
</Javascript>

In [ ]:
# Develop -> Resources -> buildLanguagesResponse.js -> XML:
var payload = context.getVariable("response.content");
var payloadObj = JSON.parse(payload);
var newPayload = JSON.stringify(payloadObj.data.languages);
context.setVariable("response.content", newPayload);

In [ ]:
# ... Save -> Deploy
# apigee-proxy@qwiklabs-gcp-02-499b66e75c7f.iam.gserviceaccount.com

In [ ]:
# SSH

curl -i -k -X GET "https://eval.example.com/translate/v1/languages"

In [ ]:
curl -i -k -X POST "https://eval.example.com/translate/v1?lang=de" -H "Content-Type:application/json" -d '{ "text": "Hello world!" }'

In [ ]:
curl -i -k -X POST "https://eval.example.com/translate/v1" -H "Content-Type:application/json" -d '{ "text": "Hello world!" }'

##Task 3. Add API key verification and quota enforcement

In [ ]:
# Apigee -> API products -> + Create -> [translate-product], Environment [eval], Access [Public],
# Operations -> Add An Operation: API Proxy [translate-v1], Path [/], Methods [GET and POST], Quota [10], [1], [minute]

In [ ]:
# Apigee -> Developers -> + Developer -> ... joe@example.com

In [ ]:
# Apigee -> Apps -> + Create -> ... Credentials -> + Add Products: [translate-app], [translate-product], [joe@example.com]

In [ ]:
# Apigee -> API proxies -> [translate-v1] -> Develop -> Proxy Endpoints | default |
# PreFlow | Request +: [Security: Verify API Key] -> [VA-VerifyKey] -> XML:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<VerifyAPIKey continueOnError="false" enabled="true" name="VA-VerifyKey">
  <DisplayName>VA-VerifyKey</DisplayName>
  <Properties/>
  <APIKey ref="request.header.Key"/>
</VerifyAPIKey>

In [ ]:
# PreFlow | Request +: [Traffic Management: Quota] -> [Q-EnforceQuota] -> XML:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<Quota name="Q-EnforceQuota" type="calendar">
  <UseQuotaConfigInAPIProduct stepName="VA-VerifyKey">
    <DefaultConfig>
      <Allow>5</Allow>
      <Interval>1</Interval>
      <TimeUnit>hour</TimeUnit>
    </DefaultConfig>
  </UseQuotaConfigInAPIProduct>
  <StartTime>2022-01-01 00:00:00</StartTime>
</Quota>

In [ ]:
# ... Save -> Deploy
# apigee-proxy@qwiklabs-gcp-02-499b66e75c7f.iam.gserviceaccount.com

In [ ]:
#SSH

curl -i -k -X POST "https://eval.example.com/translate/v1?lang=de" -H "Content-Type:application/json" -d '{ "text": "Hello world!" }'

In [ ]:
curl -i -k -X POST "https://eval.example.com/translate/v1?lang=de" -H "Content-Type:application/json" -H "Key: ABC123" -d '{ "text": "Hello world!" }'


In [ ]:
curl -i -k -X POST "https://eval.example.com/translate/v1?lang=de" -H "Content-Type:application/json" -H "Key: $KEY" -d '{ "text": "Hello world!" }'


##Task 4. Add message logging

In [ ]:
# translate | Response +: [Extension: Message Logging] -> [ML-LogTranslation] -> XML:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<MessageLogging name="ML-LogTranslation">
  <CloudLogging>
    <LogName>projects/{organization.name}/logs/translate</LogName>
    <Message contentType="text/plain">{language}|{text}|{translated}</Message>
  </CloudLogging>
</MessageLogging>

In [ ]:
# ... Save -> Deploy
# apigee-proxy@qwiklabs-gcp-02-499b66e75c7f.iam.gserviceaccount.com

In [ ]:
# SSH

curl -i -k -X POST "https://eval.example.com/translate/v1?lang=de" -H "Content-Type:application/json" -H "Key: $KEY" -d '{ "text": "Hello world!" }'


##Task 5. Rewrite a backend error message

In [ ]:
# translate | Request +: [Mediation: Assign Message] -> [AM-BuildErrorResponse] -> XML:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<AssignMessage name="AM-BuildErrorResponse">
  <Set>
    <Payload contentType="application/json">{ "error": "Invalid request. Verify the lang query parameter." }</Payload>
  </Set>
</AssignMessage>

In [ ]:
# Target Endpoints | default -> XML:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<TargetEndpoint name="default">
  <Description/>
  <FaultRules>
    <FaultRule name="invalidInput">
      <Step>
        <Name>AM-BuildErrorResponse</Name>
      </Step>
      <Condition>fault.name == "ErrorResponseCode"</Condition>
    </FaultRule>
  </FaultRules>
  <PreFlow name="PreFlow">
    <Request/>
    <Response/>
  </PreFlow>
  <PostFlow name="PostFlow">
    <Request/>
    <Response/>
  </PostFlow>
  <Flows/>
  <HTTPTargetConnection>
    <URL>https://translation.googleapis.com/language/translate/v2</URL>
    <Authentication>
      <GoogleAccessToken>
        <Scopes>
          <Scope>https://www.googleapis.com/auth/cloud-translation</Scope>
        </Scopes>
      </GoogleAccessToken>
    </Authentication>
  </HTTPTargetConnection>
</TargetEndpoint>

In [ ]:
# ... Save -> Deploy
# apigee-proxy@qwiklabs-gcp-02-499b66e75c7f.iam.gserviceaccount.com

In [ ]:
# SSH

curl -i -k -X POST "https://eval.example.com/translate/v1?lang=de" -H "Content-Type:application/json" -H "Key: $KEY" -d '{ "text": "Hello world!" }'


In [ ]:
curl -i -k -X POST "https://eval.example.com/translate/v1?lang=invalid" -H "Content-Type:application/json" -H "Key: $KEY" -d '{ "text": "Hello world!" }'
